In [68]:
import sys
sys.path.append('../')
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import datetime as dt
import numpy as np
from dateutil import parser

from api.oanda_api import OandaAPI
from infrastructure.instrument_collection import instrument_collection as ic
from mongodb.mongodb import MongoDB


In [69]:
api = OandaAPI()
db = MongoDB()

In [70]:
ic.load_instruments_db()

In [71]:
pairs = []

for pair, pair_info in ic.instruments_dict.items():
    if pair_info.ins_type == 'CURRENCY' and 'USD' in pair:
        pairs.append(pair)
        

In [72]:
pairs

['GBP_USD',
 'USD_MXN',
 'USD_HUF',
 'USD_SEK',
 'USD_JPY',
 'USD_THB',
 'USD_CHF',
 'USD_SGD',
 'USD_CZK',
 'USD_CNH',
 'USD_TRY',
 'AUD_USD',
 'USD_NOK',
 'USD_CAD',
 'USD_ZAR',
 'USD_DKK',
 'USD_PLN',
 'NZD_USD',
 'USD_HKD',
 'EUR_USD']

In [73]:
data = []

for p in pairs:
    df_temp = api.get_candles_df(pair=p, granularity='D', count=400)
    df_temp['gain'] = ((df_temp['mid_c'] - df_temp['mid_o']) / df_temp['mid_c']) * 100
    df_temp['pair'] = p

    if '_USD' in p:
        df_temp['gain'] = df_temp['gain'] * -1
        
    data.append(df_temp)

df_candles = pd.concat(data)
df_candles.reset_index(drop=True, inplace=True)
df_candles['time'] = df_candles['time'].dt.date


In [74]:
df_candles.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
7975,2024-06-05,56592,1.08710,1.09021,1.08620,1.08898,1.08697,1.09014,1.08612,1.08889,1.08724,1.09028,1.08627,1.08906,-0.172639,EUR_USD
7976,2024-06-06,69167,1.08910,1.09023,1.07993,1.08017,1.08900,1.09016,1.07986,1.08006,1.08920,1.09030,1.08000,1.08028,0.826722,EUR_USD
7977,2024-06-09,58584,1.07699,1.07817,1.07328,1.07658,1.07682,1.07810,1.07321,1.07651,1.07716,1.07824,1.07335,1.07665,0.038084,EUR_USD
7978,2024-06-10,54984,1.07649,1.07736,1.07196,1.07406,1.07631,1.07729,1.07189,1.07398,1.07667,1.07743,1.07203,1.07413,0.226244,EUR_USD
7979,2024-06-11,87675,1.07395,1.08524,1.07346,1.08092,1.07346,1.08517,1.07339,1.08085,1.07444,1.08531,1.07354,1.08100,-0.644821,EUR_USD


In [75]:
calendar_data = db.query_all(MongoDB.CALENDAR_COLL)

In [76]:
df_calendar_data = pd.DataFrame.from_dict(calendar_data)

In [77]:
df_calendar_data['date'] = df_calendar_data['date'].dt.date

In [78]:
df_calendar_data_uk = df_calendar_data[df_calendar_data['country'] == 'united states'].copy()

In [79]:
df_calendar_data_uk.head()

,date,country,category,event,symbol,actual,previous,forecast
1,2023-07-03,united states,business confidence,ism manufacturing pmi,NAPMPMI,46,46.9,48
4,2023-07-05,united states,interest rate,fomc minutes,,,,
6,2023-07-06,united states,non manufacturing pmi,ism services pmi,UNITEDSTANONMANPMI,53.9,50.3,50
7,2023-07-06,united states,job offers,jolts job openings,UNITEDSTAJOBOFF,9.8M,10.32M,9.9M
9,2023-07-07,united states,non farm payrolls,non farm payrolls,NFP TCH,209K,306K,250.0K


In [80]:
df_calendar_data_uk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 248 entries, 1 to 702
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      248 non-null    object
 1   country   248 non-null    object
 2   category  248 non-null    object
 3   event     248 non-null    object
 4   symbol    248 non-null    object
 5   actual    248 non-null    object
 6   previous  248 non-null    object
 7   forecast  248 non-null    object
dtypes: object(8)
memory usage: 17.4+ KB


In [81]:
for col in ['actual', 'previous', 'forecast']:
    for sym in ['£', '$', '%', 'B', 'K', 'M']:
        df_calendar_data_uk[col] = df_calendar_data_uk[col].str.replace(sym, '')
    
    df_calendar_data_uk[col] = np.where(df_calendar_data_uk[col] == '', 0, df_calendar_data_uk[col])
    df_calendar_data_uk[col] = df_calendar_data_uk[col].astype(float)
        
        

In [82]:
date_match = {}

for orig in df_calendar_data_uk['date'].unique():
    date = orig
    
    tries = 0
    while date not in df_candles['time'].values:
        date = date + dt.timedelta(days=1)
        
        tries += 1
        
        if tries > 5:
            print(f'Dates do not match: {tries}')
            break
    
    date_match[orig] = date
        


Dates do not match: 6
Dates do not match: 6
Dates do not match: 6


In [83]:
df_calendar_data_uk['original_date'] = df_calendar_data_uk['date']
df_calendar_data_uk['date'] = [date_match[date] for date in df_calendar_data_uk['date']]

In [84]:
df_calendar_data_uk['delta_prev'] = df_calendar_data_uk['actual'] - df_calendar_data_uk['previous']
df_calendar_data_uk['delta_forecast'] = df_calendar_data_uk['actual'] - df_calendar_data_uk['forecast']

In [85]:
df_calendar_data_uk.head(3)

,date,country,category,event,symbol,actual,previous,forecast,original_date,delta_prev,delta_forecast
1,2023-07-03,united states,business confidence,ism manufacturing pmi,NAPMPMI,46.0,46.9,48.0,2023-07-03,-0.9,-2.0
4,2023-07-05,united states,interest rate,fomc minutes,,0.0,0.0,0.0,2023-07-05,0.0,0.0
6,2023-07-06,united states,non manufacturing pmi,ism services pmi,UNITEDSTANONMANPMI,53.9,50.3,50.0,2023-07-06,3.6,3.9


In [86]:
df_candles.head(3)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
0,2022-11-28,247871,1.19578,1.20646,1.19452,1.19506,1.19474,1.20635,1.19421,1.19495,1.19682,1.20657,1.19459,1.19518,0.060248,GBP_USD
1,2022-11-29,262185,1.19600,1.20872,1.19005,1.20571,1.19503,1.20861,1.18995,1.20548,1.19697,1.20882,1.19013,1.20594,-0.805335,GBP_USD
2,2022-11-30,267875,1.20629,1.23110,1.20529,1.22550,1.20522,1.23082,1.20413,1.22540,1.20736,1.23144,1.20584,1.22560,-1.567523,GBP_USD


In [87]:
candles_analysis = df_candles[['time', 'pair', 'gain']].copy()

In [88]:
candles_analysis.head(3)

,time,pair,gain
0,2022-11-28,GBP_USD,0.060248
1,2022-11-29,GBP_USD,-0.805335
2,2022-11-30,GBP_USD,-1.567523


In [89]:
merged = pd.merge(left=candles_analysis, right=df_calendar_data_uk, left_on='time', right_on='date')

In [90]:
merged.head(3)

,time,pair,gain,date,country,category,event,symbol,actual,previous,forecast,original_date,delta_prev,delta_forecast
0,2023-07-03,GBP_USD,-0.193497,2023-07-03,united states,business confidence,ism manufacturing pmi,NAPMPMI,46.0,46.9,48.0,2023-07-03,-0.9,-2.0
1,2023-07-05,GBP_USD,-0.316349,2023-07-05,united states,interest rate,fomc minutes,,0.0,0.0,0.0,2023-07-05,0.0,0.0
2,2023-07-06,GBP_USD,-0.792872,2023-07-06,united states,non manufacturing pmi,ism services pmi,UNITEDSTANONMANPMI,53.9,50.3,50.0,2023-07-06,3.6,3.9


In [91]:
merged[merged['category'] == 'inflation rate'].head(3)

,time,pair,gain,date,country,category,event,symbol,actual,previous,forecast,original_date,delta_prev,delta_forecast
9,2023-07-12,GBP_USD,-1.128214,2023-07-12,united states,inflation rate,inflation rate yoy,CPI YOY,3.0,4.0,3.2,2023-07-12,-1.0,-0.2
29,2023-08-10,GBP_USD,-0.123675,2023-08-10,united states,inflation rate,inflation rate yoy,CPI YOY,3.2,3.0,3.1,2023-08-10,0.2,0.1
49,2023-09-13,GBP_USD,0.626944,2023-09-13,united states,inflation rate,inflation rate yoy,CPI YOY,3.7,3.2,3.5,2023-09-13,0.5,0.2


In [92]:
merged['category'].unique()

array(['business confidence', 'interest rate', 'non manufacturing pmi',
       'job offers', 'non farm payrolls', 'unemployment rate',
       'core inflation rate mom', 'core inflation rate',
       'inflation rate mom', 'inflation rate',
       'producer price inflation mom', 'consumer confidence',
       'retail sales mom', 'building permits', 'durable goods orders',
       'gdp growth rate', 'core pce price index mom', 'personal income',
       'personal spending'], dtype=object)

In [93]:
cat = 'business confidence'

df_cat = merged[merged['category'] == cat].copy()

print(df_cat[df_cat['delta_prev'] >= 0]['gain'].sum())
print(df_cat[df_cat['delta_prev'] < 0]['gain'].sum())


41.419966017346134
-10.725399050152182


In [94]:
for p in pairs:
    print(p) 
    
    df_plot = df_cat[df_cat['pair'] == p]
    
    fig = px.scatter(df_plot, x='gain', y='delta_prev', trendline='ols')
    fig.show()
    

GBP_USD


USD_MXN


USD_HUF


USD_SEK


USD_JPY


...

USD_DKK


USD_PLN


NZD_USD


USD_HKD


EUR_USD
